In [1]:
# Extract Feature Set (Select Particular Columns) from entire dataset

In [2]:
# Imports

import pandas as pd
import numpy as np
from pandas import *
import os
import re
import math

#### TODO: Specifiy Data Path Based on Participant/Your Computer

In [3]:
# TODO: Specify Data Path based on participant/your computer
# Warning: this will probably take some time

file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/p8_data_raw.csv'
data = pd.read_csv(file)

/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,4,6,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.head()

,participant,session_num,session_date,timestamp,engagement,activity,skill,difficulty,aptitude,games_total,...,op_p2_pose_y22,op_p2_pose_y23,op_p2_pose_y24,op_p2_pose_y3,op_p2_pose_y4,op_p2_pose_y5,op_p2_pose_y6,op_p2_pose_y7,op_p2_pose_y8,op_p2_pose_y9
0,NaN,0.0,NaN,NaN,disengaged,NaN,NaN,NaN,0.5,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.0,NaN,NaN,disengaged,NaN,NaN,NaN,0.5,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.0,NaN,NaN,disengaged,NaN,NaN,NaN,0.5,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,0.0,NaN,NaN,disengaged,NaN,NaN,NaN,0.5,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0.0,NaN,NaN,disengaged,NaN,NaN,NaN,0.5,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Preprocessing Steps 
1. One Hot Encode
2. Transform Labels 
3. Drop Irrelevant Columns

In [5]:
# drop session 0

data = data[~(data['session_num'] == 0)]  

In [6]:
# One Hot Encoding for Skill and Difficulty

data['skill_NC'] = 0
data['skill_OS'] = 0
data['skill_EM'] = 0

data['diff_1'] = 0
data['diff_2'] = 0
data['diff_3'] = 0
data['diff_4'] = 0
data['diff_5'] = 0

data['no_game'] = 0

for i,r in data.iterrows():
    skill_col = 'skill_'
    diff_col = 'diff_'
    
    if isinstance(r['skill'], str):
        skill_col += r['skill']
        data.at[i, skill_col] = 1
    else:
        data.at[i, 'no_game'] = 1
        
    if not (math.isnan(r['difficulty'])):
        diff_col += str(int(r['difficulty']))
        data.at[i, diff_col] = 1


In [7]:
# Transform Labels into 0s and 1s

label_map = {
    'engaged': 1,
    'disengaged': 0
}

data['engagement'] = data['engagement'].map(label_map)

data = data.dropna(subset=['engagement'])
data = data[data['engagement'].notna()]

## Feature Construction

In [8]:
# Constructed Features: 
# of_gaze_distance: approximated distance from gaze to camera on x axis
# of_pose_distance: approximated distance between camera and head position 

# Notes Here: https://docs.google.com/document/d/1q5ZSRo8NgItPHMHeVJkU-_e6YFZgjRdni2M6l5zrJlc/edit?usp=sharing

data['of_gaze_distance'] = 0.0
data['of_pose_distance'] = 0.0

data['of_gaze_angle_x'] = pd.to_numeric(data['of_gaze_angle_x'], errors='coerce')
data['of_pose_Tz'] = pd.to_numeric(data['of_pose_Tz'], errors='coerce')
data['of_pose_Tx'] = pd.to_numeric(data['of_pose_Tx'], errors='coerce')
data['of_pose_Ty'] = pd.to_numeric(data['of_pose_Ty'], errors='coerce')

for i, r in data.iterrows():    
    if (r['of_success'] == 0):
        data.at[i, 'of_gaze_distance'] = np.nan
        data.at[i, 'of_pose_distance'] = np.nan
    else:
        data.at[i, 'of_gaze_distance'] = (math.tan(r['of_gaze_angle_x'])*r['of_pose_Tz']) + r['of_pose_Tx']
        data.at[i, 'of_pose_distance'] = math.sqrt(math.pow(r['of_pose_Tx'], 2) + math.pow(r['of_pose_Ty'], 2) + math.pow(r['of_pose_Tz'], 2))

## Save Selected Features

Note: keep timestamp and session_num until right before running the model in all feature sets

In [9]:
basic = ['confidence', 'success', 'engagement']

gaze = ['gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x', 'gaze_angle_y', 'gaze_distance']

pose = ['pose_Tx', 'pose_Ty', 'pose_Tz', 'pose_Rx', 'pose_Ry', 'pose_Rz', 'pose_distance']

action_units = ['AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06_c', 'AU07_c', 'AU09_c', 'AU10_c', 'AU12_c', 'AU14_c', 'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c', 'AU28_c', 'AU45_c']

hand_picked = basic + gaze + pose + action_units

for i,c in enumerate(hand_picked):
    hand_picked[i] = 'of_' + c
hand_picked += ['op_Number of People', 'timestamp', 'session_num', 'participant']

performance = ['games_total', 'games_session', 'mistakes_total', 'mistakes_session', 'mistakes_game', 'ts_robot_talked', 'ts_game_start', 'ts_attempt', 'aptitude']

game_info = ['diff_1', 'diff_2', 'diff_3', 'diff_4', 'diff_5', 'skill_NC', 'skill_OS', 'skill_EM', 'no_game']

hand_picked += performance + game_info

FS = data[hand_picked]

In [1]:
# Save Min Max 
'''
min_track = {}

max_track = {}

path_data = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/min_track.csv'
min_df = pd.read_csv(path_data)

path_data = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/max_track.csv'
max_df = pd.read_csv(path_data)

for c in FS.columns:
    print(c)
        
    try: 
        min_val = FS.loc[:, c].min()
    except:
        FS.loc[:, c] = pd.to_numeric(FS.loc[:,c], errors='coerce')
        min_val = FS.loc[:, c].min()
        
    max_val = FS.loc[:, c].max()
    
    min_track[c] = min_val
    max_track[c] = max_val

min_df = min_df.append(min_track, ignore_index=True)
max_df = max_df.append(max_track, ignore_index=True)
'''

"\nmin_track = {}\n\nmax_track = {}\n\npath_data = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/min_track.csv'\nmin_df = pd.read_csv(path_data)\n\npath_data = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/max_track.csv'\nmax_df = pd.read_csv(path_data)\n\nfor c in FS.columns:\n    print(c)\n        \n    try: \n        min_val = FS.loc[:, c].min()\n    except:\n        FS.loc[:, c] = pd.to_numeric(FS.loc[:,c], errors='coerce')\n        min_val = FS.loc[:, c].min()\n        \n    max_val = FS.loc[:, c].max()\n    \n    min_track[c] = min_val\n    max_track[c] = max_val\n\nmin_df = min_df.append(min_track, ignore_index=True)\nmax_df = max_df.append(max_track, ignore_index=True)\n"

### Write to File

In [11]:
path_data = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/p8_FS.csv'

FS.to_csv(path_data, index=False)

In [12]:
#path_data = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/min_track.csv'

#min_df.to_csv(path_data, index=False)

In [13]:
#path_data = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/max_track.csv'

#max_df.to_csv(path_data, index=False)